### Second try to get this automatic finite difference generator

In [1]:
from sympy import *

### First set up a test system

In [19]:
template = Matrix([[1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, -1, 1, 1],
                   [1, 1, 1, 1, 1],
                   [1, 1, 1, 1, 1]])
order = 2
partial = (3, 1)
deg = partial[0] + partial[1]

m, n = template.shape
k = deg + order
x, y = symbols('x y')

### Order gridpoints as a vector, make a list consisting of tuples $(i, j)$ where $i$ is the $x$ offset and $j$ is the $y$ offset

In [20]:
if not m % 2 or not n % 2:
    raise ValueError("Template dimensions must be odd")         
if m is not n:
    raise ValueError("Template matrix must be square")
    
# Find center of template, marked by -1
center = 0
for i in range(m):
    for j in range(n):
        if template[i, j] == -1:
            center = (i, j)
if not center:
    raise ValueError("Did not set center of template")

# Make list of offsets from center in the template
# Note that it's in ij index convention
offsets = []
for i in range(m):
    for j in range(n):
        if template[i, j] == 1:
            offsets.append((j - center[1], center[0] - i))

### Generate monomials in graded lexicographical order, see which place in the Taylor series the desired partial is

In [21]:
f = exp(x)*exp(y)
q = expand( f.series(x, 0, k).removeO().series(y, 0, k).removeO() )
q = sum(term for term in q.args if degree(term, x) + degree(term, y) < k)
q -= 1
q = poly(q, x, y)
place = q.monoms(order='grlex').index(partial)

### Generate matrix corresponding to Taylor series system of equations

In [12]:
M = zeros(len(offsets), q.length())
offset_count = 0
for (i, j) in offsets:
    f = exp(i*x)*exp(j*y)
    p = expand( f.series(x, 0, k).removeO().series(y, 0, k).removeO() )
    p = sum(term for term in p.args if degree(term, x) + degree(term, y) < k)
    p -= 1
    p = poly(p, x, y)

    term_count = 0
    for deg in q.monoms(order='grlex'):
        M[offset_count, term_count] = p.coeff_monomial(deg)
        term_count += 1
    offset_count += 1

In [13]:
M_inv = M.pinv()
deriv_vec = zeros(q.length(), 1)
deriv_vec[place] = 1
if M_inv*M*deriv_vec == deriv_vec:
    stencil = template
    idx_count = 0
    coeff_sum = 0
    for i in range(m):
        for j in range(n):
            if template[i, j] == 1:
                stencil[i, j] = M_inv[place, idx_count]
                coeff_sum += M_inv[place, idx_count]
                idx_count += 1
    stencil[center[0], center[1]] = -coeff_sum
else:
    print("Could not make a stencil")

In [14]:
M_inv

Matrix([
[  1452/134015,  -2904/134015,            0,  2904/134015,  -1452/134015,  -5808/134015,  11616/134015,             0, -11616/134015,  5808/134015,  -7388/134015, 16876/134015, -16876/134015,  7388/134015,  -5808/134015, 11616/134015,            0, -11616/134015,   5808/134015,  1452/134015,  -2904/134015,             0,   2904/134015,  -1452/134015],
[          1/5,          -4/5,          6/5,         -4/5,           1/5,          1/10,          -2/5,           3/5,          -2/5,         1/10,             0,            0,             0,            0,         -1/10,          2/5,         -3/5,           2/5,         -1/10,         -1/5,           4/5,          -6/5,           4/5,          -1/5],
[         -1/7,           2/7,            0,         -2/7,           1/7,          1/14,          -1/7,             0,           1/7,        -1/14,           1/7,         -2/7,           2/7,         -1/7,          1/14,         -1/7,            0,           1/7,         -1/14,     

In [15]:
stencil

Matrix([
[-1/10,   1/5, 0,  -1/5,  1/10],
[-1/20,  1/10, 0, -1/10,  1/20],
[    0,     0, 0,     0,     0],
[ 1/20, -1/10, 0,  1/10, -1/20],
[ 1/10,  -1/5, 0,   1/5, -1/10]])

### Want to let users know what order monomials will be in

In [99]:
f = exp(x)*exp(y)
q = expand( f.series(x, 0, k).removeO().series(y, 0, k).removeO() )
q = sum(term for term in q.args if degree(term, x) + degree(term, y) < k)
q -= 1
q = poly(q, x, y)

q.monoms(order='grlex')

[(5, 0),
 (4, 1),
 (3, 2),
 (2, 3),
 (1, 4),
 (0, 5),
 (4, 0),
 (3, 1),
 (2, 2),
 (1, 3),
 (0, 4),
 (3, 0),
 (2, 1),
 (1, 2),
 (0, 3),
 (2, 0),
 (1, 1),
 (0, 2),
 (1, 0),
 (0, 1)]